In [1]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod("media-microservices")
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.244.0.4	media-microservices	cast-info-memcached-56b648cb4-274nc
10.244.0.34	media-microservices	cast-info-mongodb-566f5887c-vqzp6
10.244.0.18	media-microservices	cast-info-service-7ccdbff8-zf45m
10.244.0.32	media-microservices	compose-review-memcached-598d9dc6cd-cq2dm
10.244.0.5	media-microservices	compose-review-service-76ccb99f59-dgdcn
10.244.0.7	media-microservices	jaeger-58bb7ff659-vs6dh
10.244.0.22	media-microservices	movie-id-memcached-d5c78ff8d-qv8sz
10.244.0.12	media-microservices	movie-id-mongodb-75479f887f-ktrb5
10.244.0.8	media-microservices	movie-id-service-7d8d47875b-bdz9c
10.244.0.23	media-microservices	movie-info-memcached-6d87c6f986-x25jz
10.244.0.19	media-microservices	movie-info-mongodb-7667c59cb8-tmhrf
10.244.0.16	media-microservices	movie-info-service-558585d447-55dfn
10.244.0.3	media-microservices	movie-review-mongodb-7d4cdd95d8-bw2fx
10.244.0.10	media-microservices	movie-review-redis-6d9595df9f-7bb7z
10.244.0.13	media-microservices	

In [5]:
api_client = client.ApiClient()
current_node = 0
node_names = ["minikube"]

In [10]:
api_client = client.ApiClient()
current_node = 0
node_names = ["minikube"]
response = api_client.call_api(f"/api/v1/nodes/{node_names[current_node]}/proxy/metrics/resource", "GET", response_type="str")
data = str(response[0])
rows = data.split("\n")
rows

['# HELP container_cpu_usage_seconds_total [ALPHA] Cumulative cpu time consumed by the container in core-seconds',
 '# TYPE container_cpu_usage_seconds_total counter',
 'container_cpu_usage_seconds_total{container="cast-info-memcached",namespace="media-microservices",pod="cast-info-memcached-56b648cb4-274nc"} 1.863737 1709119445656',
 'container_cpu_usage_seconds_total{container="cast-info-mongodb",namespace="media-microservices",pod="cast-info-mongodb-566f5887c-vqzp6"} 67.178791 1709119445662',
 'container_cpu_usage_seconds_total{container="cast-info-service",namespace="media-microservices",pod="cast-info-service-7ccdbff8-zf45m"} 2.303045 1709119446795',
 'container_cpu_usage_seconds_total{container="compose-review-memcached",namespace="media-microservices",pod="compose-review-memcached-598d9dc6cd-cq2dm"} 69.726172 1709119445446',
 'container_cpu_usage_seconds_total{container="compose-review-service",namespace="media-microservices",pod="compose-review-service-76ccb99f59-dgdcn"} 311.35

In [16]:
class Pods:
    def __init__(self):
        self.data = {}

    def __str__(self):
        return str(data)
    
    def __repr__(self):
        return str(data)
    
    def add_cpu(self, name, namespace, timestamp, value):
        try:
            self.data[name]["cpu"] = value
            self.data[name]["cpu_timestamp"] = timestamp
        except KeyError:
            self.data[name] = {"namespace": namespace, "pod": name, "cpu": value, "cpu_timestamp": timestamp}

    def add_memory(self, name, namespace, timestamp, value):
        try:
            self.data[name]["memory"] = value
            self.data[name]["memory_timestamp"] = timestamp
        except KeyError:
            self.data[name] = {"namespace": namespace, "pod": name, "memory": value, "memory_timestamp": timestamp}

class Nodes:
    def __init__(self):
        self.data = {}

    def __str__(self):
        return str(data)
    
    def __repr__(self):
        return str(data)
    
    def add_cpu(self, name, timestamp, value):
        try:
            self.data[name]["cpu"] = value
            self.data[name]["cpu_timestamp"] = timestamp
        except KeyError:
            self.data[name] = {"cpu": value, "cpu_timestamp": timestamp}

    def add_memory(self, name, timestamp, value):
        try:
            self.data[name]["memory"] = value
            self.data[name]["memory_timestamp"] = timestamp
        except KeyError:
            self.data[name] = {"memory": value, "memory_timestamp": timestamp}

class Containers:
    def __init__(self):
        self.data = {}

    def __str__(self):
        return str(data)
    
    def __repr__(self):
        return str(data)
    
    def add_cpu(self, name, namespace, pod, timestamp, value):
        try:
            self.data[pod]["cpu"] = value
            self.data[pod]["cpu_timestamp"] = timestamp
        except KeyError:
            self.data[pod] = {"namespace": namespace, "pod": pod, "container": name, "cpu": value, "cpu_timestamp": timestamp}

    def add_memory(self, name, namespace, pod, timestamp, value):
        try:
            self.data[pod]["memory"] = value
            self.data[pod]["memory_timestamp"] = timestamp
        except KeyError:
            self.data[pod] = {"namespace": namespace, "pod": pod, "container": name, "memory": value, "memory_timestamp": timestamp}

In [11]:
data = str(response[0])
rows = data.split("\n")

# Remove comments and empty lines
rows = [row for row in rows if not row.startswith("#") and row.strip() != ""]

# Split lines from rows
rows = [row.split() for row in rows]


# Create a dictionary with the data
pods = {}
containers = {}
nodes = {}
error_count = 0

def process_pod_or_container(row: list[str]):
    raw_name = row[0]
    value = row[1]
    timestamp = row[2]
    raw_type = raw_name.split("{")[0]
    pod = raw_name.split("\"")[-2].split("\"")[-1]
    namespace = raw_name.split("\"")[-4].split("\"")[-1]
    container = raw_name.split("\"")[1].split("\"")[0]
    match raw_type:
        case "container_cpu_usage_seconds_total":
            try:
                containers[pod]["cpu"] = value
                containers[pod]["cpu_timestamp"] = timestamp
            except KeyError:
                containers[pod] = {"namespace": namespace, "container": container, "pod": pod, "cpu": value, "cpu_timestamp": timestamp}
        case "container_memory_usage_bytes":
            try:
                containers[pod]["memory"] = value
                containers[pod]["memory_timestamp"] = timestamp
            except KeyError:
                containers[pod] = {"namespace": namespace, "container": container, "pod": pod, "memory": value, "memory_timestamp": timestamp}
        case "pod_memory_working_set_bytes":
            try:
                pods[pod]["memory"] = value
                pods[pod]["memory_timestamp"] = timestamp
            except KeyError:
                pods[pod] = {"namespace": namespace, "pod": pod, "memory": value, "memory_timestamp": timestamp}
        case "pod_cpu_usage_seconds_total":
            try:
                pods[pod]["cpu"] = value
                pods[pod]["cpu_timestamp"] = timestamp
            except KeyError:
                pods[pod] = {"namespace": namespace, "pod": pod, "cpu": value, "cpu_timestamp": timestamp}


def process_node(row):
    raw_type = row[0]
    value = row[1]
    timestamp = row[2]
    match raw_type:
        case "node_memory_working_set_bytes":
            try:
                nodes[node_names[current_node]]["memory"] = value 
            except KeyError:
                nodes[node_names[current_node]] = {"memory": value, "timestamp": timestamp}
        case "node_cpu_usage_seconds_total":
            try:
                nodes[node_names[current_node]]["cpu"] = value 
            except KeyError:
                nodes[node_names[current_node]] = {"cpu": value, "timestamp": timestamp}



for row in rows:
    try:
        if row[0] == "scrape_error":
            error_count = row[1]
        elif row[0][:4] == "node":
            process_node(row)
        else:
            process_pod_or_container(row)
            
    except IndexError:
        print("Error:",row)


In [42]:
containers

{'cast-info-memcached-56b648cb4-274nc': {'namespace': 'media-microservices',
  'container': 'cast-info-memcached',
  'pod': 'cast-info-memcached-56b648cb4-274nc',
  'cpu': '1.831069',
  'cpu_timestamp': '1709119067945',
  'memory_timestamp': '1709119059484'},
 'cast-info-mongodb-566f5887c-vqzp6': {'namespace': 'media-microservices',
  'container': 'cast-info-mongodb',
  'pod': 'cast-info-mongodb-566f5887c-vqzp6',
  'cpu': '65.995307',
  'cpu_timestamp': '1709119068060',
  'memory_timestamp': '1709119066777'},
 'cast-info-service-7ccdbff8-zf45m': {'namespace': 'media-microservices',
  'container': 'cast-info-service',
  'pod': 'cast-info-service-7ccdbff8-zf45m',
  'cpu': '2.297739',
  'cpu_timestamp': '1709119069107',
  'memory_timestamp': '1709119053932'},
 'compose-review-memcached-598d9dc6cd-cq2dm': {'namespace': 'media-microservices',
  'container': 'compose-review-memcached',
  'pod': 'compose-review-memcached-598d9dc6cd-cq2dm',
  'cpu': '69.678921',
  'cpu_timestamp': '17091190680

In [5]:
pods

{'coredns-5dd5756b68-xpdq5': {'cpu': '11.362751',
  'namespace': 'kube-system',
  'timestamp': '1709109823228',
  'pod': 'coredns-5dd5756b68-xpdq5',
  'memory': '3.0539776e+07'},
 'etcd-minikube': {'cpu': '112.087974',
  'namespace': 'kube-system',
  'timestamp': '1709109815866',
  'pod': 'etcd-minikube',
  'memory': '8.2300928e+07'},
 'kube-apiserver-minikube': {'cpu': '288.816945',
  'namespace': 'kube-system',
  'timestamp': '1709109824472',
  'pod': 'kube-apiserver-minikube',
  'memory': '2.83435008e+08'},
 'kube-controller-manager-minikube': {'cpu': '85.602686',
  'namespace': 'kube-system',
  'timestamp': '1709109816513',
  'pod': 'kube-controller-manager-minikube',
  'memory': '6.4032768e+07'},
 'kube-proxy-ht6xb': {'cpu': '2.988617',
  'namespace': 'kube-system',
  'timestamp': '1709109825232',
  'pod': 'kube-proxy-ht6xb',
  'memory': '3.4811904e+07'},
 'kube-scheduler-minikube': {'cpu': '17.420849',
  'namespace': 'kube-system',
  'timestamp': '1709109815429',
  'pod': 'kube-s

In [25]:
import pandas as pd


df = pd.DataFrame(pods.values())

41
41


In [3]:
# Delete the csv files
import os
os.remove("pods.csv")
os.remove("error_count.csv")
os.remove("containers.csv")
os.remove("nodes.csv")

In [15]:
import sched, time, os
import pandas as pd

# Create empty csv files

def do_something(scheduler): 
    # schedule the next call first
    scheduler.enter(5, 1, do_something, (scheduler,))
    print("Doing stuff...")

    # Create a dictionary with the data
    pods = {}
    containers = {}
    nodes = {}
    error_count = 0
    response = api_client.call_api(f"/api/v1/nodes/{node_names[current_node]}/proxy/metrics/resource", "GET", response_type="str")
    data = str(response[0])
    rows = data.split("\n")

    # Remove comments and empty lines
    rows = [row for row in rows if not row.startswith("#") and row.strip() != ""]

    # Split lines from rows
    rows = [row.split() for row in rows]

    for row in rows:
        try:
            if row[0] == "scrape_error":
                error_count = row[1]
            elif row[0][:4] == "node":
                process_node(row)
            else:
                process_pod_or_container(row)
                
        except IndexError:
            print("Error:",row)

    pods_df = pd.DataFrame(pods.values())
    pods_df.to_csv("pods.csv", mode='a', header=not os.path.exists("pods.csv"), index=False)

    containers_df = pd.DataFrame(containers.values())
    containers_df.to_csv("containers.csv", mode='a', header=not os.path.exists("containers.csv"), index=False)

    nodes_df = pd.DataFrame(nodes.values())
    nodes_df.to_csv("nodes.csv", mode='a', header=not os.path.exists("nodes.csv"), index=False)

    error_count_df = pd.DataFrame({"error_count": [error_count], "timestamp": [time.time()]})
    error_count_df.to_csv("error_count.csv", mode='a', header=not os.path.exists("error_count.csv"), index=False)

    print("Done")

my_scheduler = sched.scheduler(time.time, time.sleep)
my_scheduler.enter(0, 1, do_something, (my_scheduler,))
my_scheduler.run()

Doing stuff...


UnboundLocalError: cannot access local variable 'pods_df' where it is not associated with a value